In [1]:
!pip3 install robustness

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 29.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 17.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.9/101.9 kB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.4/119.4 kB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.6/207.6 kB 95.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 147.6 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 184.0 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 174.2 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 

In [2]:
with open("/usr/local/lib/python3.10/dist-packages/robustness/imagenet_models/__init__.py") as rf:
    contents = rf.read()
    contents = contents.replace("from .vgg import *", "#")
    contents = contents.replace("from .alexnet import *", "#")
    contents = contents.replace("from .squeezenet import *", "#")
    with open("/usr/local/lib/python3.10/dist-packages/robustness/imagenet_models/__init__.py", "w") as wf:
        wf.write(contents)

In [3]:
with open("/usr/local/lib/python3.10/dist-packages/robustness/train.py") as rf:
    contents = rf.read()
    target_string = '''    for i, (inp, target) in iterator:
       # measure data loading time
        target = target.cuda(non_blocking=True)
        output, final_inp = model(inp, target=target, make_adv=adv,
                                  **attack_kwargs)
        loss = train_criterion(output, target)'''
    replace_string = '''    import random
    import copy
    for i, (inp, target) in iterator:
       # measure data loading time
        target = target.cuda(non_blocking=True)
        atk_kwargs = {}
        atk = random.randint(1, 3)
        if atk == 1:
            atk_kwargs = copy.deepcopy(attack_kwargs)
        output, final_inp = model(inp, target=target, make_adv=adv,
                                  **attack_kwargs)
        loss = train_criterion(output, target)'''
    contents = contents.replace(target_string, replace_string)
    with open("/usr/local/lib/python3.10/dist-packages/robustness/train.py", "w") as wf:
        wf.write(contents)

In [5]:
import torchvision
from robustness import model_utils, datasets, train, defaults
from robustness.datasets import CIFAR
import torch
import torch.nn as nn
#from r50cosine import R50Cosine

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
def set_seeds(seed):
    torch.manual_seed(seed)
    import random
    random.seed(seed)
    import numpy as np
    np.random.seed(seed)
    torch.mps.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic=True
set_seeds(0)
from cox.utils import Parameters
import cox.store

# Hard-coded dataset, architecture, batch size, workers
ds = CIFAR('/path/to/cifar')
#ds = ImageNet('/path/to/imagenet')
m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/cifar_linf_8.pt")
#m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="workspace/resnet50_linf_eps8.0.ckpt")
initials = torch.randn([2048, 10])
class CosineLayer(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(data=initials.to(device), requires_grad=True)
        self.bias = nn.Parameter(data=torch.randn(self.out_features))
    def forward(self, x):
        #print(torch.norm(x, dim=1))
        x = x / torch.norm(x, dim=1, keepdim=True)
        params = self.weight / torch.norm(self.params, dim=0)
        #concatenated = torch.stack([self.similarity(torch.stack([x[n] for m in range(self.out_features)], dim=1), self.params) for n in torch.arange(x.shape[0])], dim=0)
        #return concatenated
        return torch.matmul(x, params)
#print(m)
train_loader, val_loader = ds.make_loaders(batch_size=128, workers=8)

=> loading checkpoint '/workspace/cifar_linf_8.pt'
=> loaded checkpoint '/workspace/cifar_linf_8.pt' (epoch 153)
==> Preparing dataset cifar..


100%|██████████| 170498071/170498071 [00:02<00:00, 73643759.00it/s]


Extracting /path/to/cifar/cifar-10-python.tar.gz to /path/to/cifar
Files already downloaded and verified


In [7]:
dir(ds)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'custom_class',
 'custom_class_args',
 'data_path',
 'ds_name',
 'get_model',
 'label_mapping',
 'make_loaders',
 'mean',
 'num_classes',
 'override_args',
 'std',
 'transform_test',
 'transform_train']

In [66]:
train_loader_prep, val_loader_prep = ds.make_loaders(batch_size=1, workers=8)

==> Preparing dataset cifar..
Files already downloaded and verified
Files already downloaded and verified


In [62]:
device = torch.device('cuda')
test_tensor = torch.randn([1, 3, 32, 32]).to(device)
m = m.to(device)
m(test_tensor)[0]

tensor([[-0.0388,  0.0435,  0.0112, -0.0292,  0.0072, -0.0178, -0.0195, -0.0265,
          0.0240,  0.0147]], device='cuda:0', grad_fn=<MmBackward0>)

In [101]:
class CosineLayer(nn.Module):
    def __init__(self, in_features: int, out_features: int):
        super().__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(data=initials.to(device), requires_grad=True)
        self.bias = nn.Parameter(data=torch.randn(self.out_features), requires_grad=False)
    def forward(self, x):
        #print(torch.norm(x, dim=1))
        weight = torch.permute(self.weight, (1, 0))
        x = x / torch.norm(x, dim=1, keepdim=True)
        weight = weight / torch.norm(weight, dim=0)
        #concatenated = torch.stack([self.similarity(torch.stack([x[n] for m in range(self.out_features)], dim=1), self.params) for n in torch.arange(x.shape[0])], dim=0)
        #return concatenated
        return torch.matmul(x, weight)

In [11]:
import torch
import torch.nn as nn
device = torch.device('cuda')
#test_tensor = torch.randn([1, 3, 32, 32]).to(device)
initials = torch.randn([10, 2048])
linearlayer = nn.Linear(2048, 10)
m.model.linear = linearlayer
m.attacker.model.linear = linearlayer
m = m.to(device)
m(test_tensor)[0]

NameError: name 'test_tensor' is not defined

In [103]:
initials = torch.randn([2048, 10])
m.model.linear = nn.Identity()
lists = [[-1, -1, -1, -1, -1, -1, -1, -1, -1, -1] for x in range(1)]
for (i, data) in enumerate(train_loader):
    #print(data[0].shape, data[1])
    j = 0
    while sum([1 if x == -1 else 0 for x in lists[j]]) != 0:
        for k in range(128):
            i = 0
            #print(data[1])
            #index = random.randint(0, 49999)
            label = data[1][k].item()
            #print(k)
            if lists[i][label] == -1:
                print(f"Doing label {label}")
                lists[i][label] = 1
                outputs = m(data[0][k].reshape([1, 3, 32, 32]).to(device))[0].detach()
                for item in range(2048):
                    initials[item][label] = outputs[0][item].detach()
initials = torch.permute(initials, (1, 0))
m.model.linear = CosineLayer(2048, 10)
m.attacker.model.linear = CosineLayer(2048, 10)

Doing label 0
Doing label 2
Doing label 8
Doing label 9
Doing label 3
Doing label 6
Doing label 4
Doing label 5
Doing label 1
Doing label 7


In [105]:
for (i, data) in enumerate(train_loader): # not well separated at all, the 10 classes seem bunched together...
    test_tensor = data[0][67].unsqueeze(0).to(device)
    label = data[1][67]
    print(m(test_tensor)[0])
    print(label)
    break

tensor([[0.9720, 0.9371, 0.9895, 0.9827, 0.9788, 0.9855, 0.9666, 0.9807, 0.9792,
         0.9540]], device='cuda:0', grad_fn=<MmBackward0>)
tensor(2)


In [8]:
!pip3 install tables
!pip3 install typing_extensions==4.7.1 --upgrade


[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip
  Attempting uninstall: typing_extensions
    Found existing installation: typing_extensions 4.4.0
    Uninstalling typing_extensions-4.4.0:
      Successfully uninstalled typing_extensions-4.4.0

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [48]:
m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/cifar_linf_8.pt")

=> loading checkpoint '/workspace/cifar_linf_8.pt'
=> loaded checkpoint '/workspace/cifar_linf_8.pt' (epoch 153)


In [26]:
baseline_model, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/cifar_linf_8.pt")

=> loading checkpoint '/workspace/cifar_linf_8.pt'
=> loaded checkpoint '/workspace/cifar_linf_8.pt' (epoch 153)


In [60]:
# Hard-coded base parameters
m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/cifar_linf_8.pt")
#m.model.linear = nn.Linear(2048, 10)
#m.attacker.model.linear = nn.Linear(2048, 10)
%mkdir train_out
clean_train_kwargs = {
    'out_dir': "train_out",
    'epochs': 4,
    'lr': 0.002, # at this checkpoint don't use an excessively high learning rate (0.001 is usually enough), or else it overfits to that threat model
    'adv_train': 0,
    'adv_eval': 1,
    'constraint': 'inf',
    'eps': 8/255,
    'attack_lr': 1.5,
    'attack_steps': 20
}
clean_train_args = Parameters(clean_train_kwargs)

# Fill whatever parameters are missing from the defaults
clean_train_args = defaults.check_and_fill_args(clean_train_args,
                        defaults.TRAINING_ARGS, CIFAR)
clean_train_args = defaults.check_and_fill_args(clean_train_args,
                        defaults.PGD_ARGS, CIFAR)
adv_train_kwargs = {
    'out_dir': "train_out",
    'epochs': 1,
    'lr': 0.002, # at this checkpoint don't use an excessively high learning rate (0.001 is usually enough), or else it overfits to that threat model
    'adv_train': 1,
    'adv_eval': 1,
    'constraint': 'inf',
    'eps': 8/255,
    'attack_lr': 2/255,
    'attack_steps': 10
}
adv_train_args = Parameters(adv_train_kwargs)

# Fill whatever parameters are missing from the defaults
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.TRAINING_ARGS, CIFAR)
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.PGD_ARGS, CIFAR)

# Train a model
try:
    m = train.train_model(adv_train_args, m, (train_loader, val_loader))
except AssertionError:
    m = train.train_model(adv_train_args, m.module, (train_loader, val_loader))
#m = train.train_model(adv_train_args, m.module, (train_loader, val_loader))
#train.train_model(adv_train_args, m, (train_loader, val_loader))

=> loading checkpoint '/workspace/cifar_linf_8.pt'
=> loaded checkpoint '/workspace/cifar_linf_8.pt' (epoch 153)
mkdir: cannot create directory ‘train_out’: File exists


Train Epoch:0 | Loss 0.8774 | AdvPrec1 65.712 | AdvPrec5 97.766 | Reg term: 0.0 ||: 100%|██████████| 391/391 [02:41<00:00,  2.42it/s]
Val Epoch:0 | Loss 0.4324 | NatPrec1 87.020 | NatPrec5 99.480 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:01<00:00, 48.11it/s]
Val Epoch:0 | Loss 1.2626 | AdvPrec1 54.460 | AdvPrec5 95.300 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:29<00:00,  2.66it/s]


In [61]:
from robustness.train import eval_model
#m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/cifar_linf_8.pt")
adv_train_kwargs = {
    'out_dir': "train_out",
    'epochs': 1,
    'lr': 0.002, # at this checkpoint don't use an excessively high learning rate (0.001 is usually enough), or else it overfits to that threat model
    'adv_train': 1,
    'adv_eval': 1,
    'constraint': 'inf',
    'eps': 8/255,
    'attack_lr': 2/255,
    'attack_steps': 20
}
adv_train_args = Parameters(adv_train_kwargs)

# Fill whatever parameters are missing from the defaults
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.TRAINING_ARGS, CIFAR)
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.PGD_ARGS, CIFAR)
try:
    eval_model(adv_train_args, m.module, val_loader, None)
except AttributeError:
    eval_model(adv_train_args, m, val_loader, None)
adv_train_kwargs = {
    'out_dir': "train_out",
    'epochs': 1,
    'lr': 0.002, # at this checkpoint don't use an excessively high learning rate (0.001 is usually enough), or else it overfits to that threat model
    'adv_train': 1,
    'adv_eval': 1,
    'constraint': 'inf',
    'eps': 8/255,
    'attack_lr': 2/255,
    'attack_steps': 100
}
adv_train_args = Parameters(adv_train_kwargs)

# Fill whatever parameters are missing from the defaults
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.TRAINING_ARGS, CIFAR)
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.PGD_ARGS, CIFAR)
try:
    eval_model(adv_train_args, m.module, val_loader, None)
except AttributeError:
    eval_model(adv_train_args, m, val_loader, None)

Val Epoch:0 | Loss 0.4324 | NatPrec1 87.020 | NatPrec5 99.480 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:01<00:00, 47.67it/s]
Val Epoch:0 | Loss 1.2875 | AdvPrec1 53.700 | AdvPrec5 95.380 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:56<00:00,  1.41it/s]
Val Epoch:0 | Loss 0.4324 | NatPrec1 87.020 | NatPrec5 99.480 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:01<00:00, 49.29it/s]
Val Epoch:0 | Loss 1.2978 | AdvPrec1 53.310 | AdvPrec5 95.420 | Reg term: 0.0 ||: 100%|██████████| 79/79 [04:27<00:00,  3.39s/it]


In [7]:
!pip3 install torchattacks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 77.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 33.9 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.25.0 requires requests>=2.31, but you have requests 2.25.1 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip i

In [9]:
net = None
def evaluate_attacked_model(loader, phase, atk, limit=5000, model=net):
    correct = 0
    total = 0
    counter = 0
    total_loss = 0.0
    for data in loader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        #print(labels)
        net.train()
        images = atk(images, labels)
        net.eval()
        outputs = model(images.to(device))
        #print(outputs.shape)
        # the class with the highest energy is what we choose as prediction
        #print(outputs.data)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total >= limit:
            break
    print(f'{phase} accuracy: {100 * correct / total} ({correct} / {total}) %')
    return correct, total_loss / math.ceil(limit / 50)

In [47]:
import torchattacks
set_seeds(0)
device = torch.device('cuda')
model = nn.Sequential(baseline_model.normalizer, baseline_model.model).to(device)
net = nn.Sequential(m.module.normalizer, m.module.model).to(device)
pgd10 = torchattacks.PGD(net, steps=20)
pgd10 = torchattacks.VANILA(net)
criterion = nn.CrossEntropyLoss()
import math
evaluate_attacked_model(val_loader, "test", pgd10, limit=10000, model=net)

test accuracy: 86.96 (8696 / 10000) %


(8696, 0.17230087623000145)

In [ ]:
# torchattacks baseline: 54.27% PGD-10, 53.01% PGD-20
# torchattacks baseline + 1-epoch PGD5: 53.04% PGD-20
# torchattacks net on baseline: 54.70% PGD-20

In [56]:
torch.save(m.state_dict(), "53.570_pgd20.pth")

In [11]:
model_state_dict_backup = m.state_dict()

In [14]:
m.load_state_dict(model_state_dict_backup)

<All keys matched successfully>

In [12]:
state_dict = torch.load("/workspace/cifar_linf_8.pt")["state_dict"]
model_state_dict = m.state_dict()
import copy
new_state_dict = copy.deepcopy(state_dict)
for item in state_dict.keys():
    if "attacker" in item:
        model_state_dict[item] = state_dict[item]
m.load_state_dict(model_state_dict)

<All keys matched successfully>

In [57]:
eval_train_kwargs = {
    'out_dir': "train_out",
    'epochs': 1,
    'lr': 0, # at this checkpoint don't use an excessively high learning rate (0.001 is usually enough), or else it overfits to that threat model
    'adv_train': 0,
    'adv_eval': 1,
    'constraint': 'inf',
    'eps': 8/255,
    'attack_lr': 2/255,
    'attack_steps': 20
}
adv_train_args = Parameters(adv_train_kwargs)

# Fill whatever parameters are missing from the defaults
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.TRAINING_ARGS, CIFAR)
adv_train_args = defaults.check_and_fill_args(adv_train_args,
                        defaults.PGD_ARGS, CIFAR)
m = train.train_model(adv_train_args, m, (train_loader, val_loader))

=> loading checkpoint '/workspace/cifar_linf_8.pt'
=> loaded checkpoint '/workspace/cifar_linf_8.pt' (epoch 153)


Train Epoch:0 | Loss 0.9503 | AdvPrec1 61.016 | AdvPrec5 97.578 | Reg term: 0.0 ||:   3%|▎         | 10/391 [00:07<04:59,  1.27it/s]


KeyboardInterrupt: 

In [13]:
!pip3 install torchattacks

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.0/142.0 kB 1.3 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.2/61.2 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.4 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.4
    Uninstalling idna-3.4:
      Successfully uninstalled idna-3.4
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyterlab-server 2.25.0 requires requests>=2.31, but you have requests 2.25.1 which is incompatible.

[notice] A new release of pip is available: 23.3.1 -> 25.0
[notice] To update, run: python -m 

In [112]:
for item in range(10):
    m = train.train_model(clean_train_args, m.module, (train_loader, val_loader))
    # TODO: do a torchattacks PGD evaluation here
    m = train.train_model(adv_train_args, m.module, (train_loader, val_loader))
    # TODO: do a torchattacks PGD evaluation here

Train Epoch:0 | Loss 2.2048 | NatPrec1 81.294 | NatPrec5 99.656 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:21<00:00, 17.97it/s]
Val Epoch:0 | Loss 2.1968 | NatPrec1 78.040 | NatPrec5 99.250 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:01<00:00, 47.87it/s]
Val Epoch:0 | Loss 2.2261 | AdvPrec1 51.480 | AdvPrec5 91.830 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:54<00:00,  1.45it/s]
Train Epoch:0 | Loss 2.2108 | AdvPrec1 59.958 | AdvPrec5 95.432 | Reg term: 0.0 ||: 100%|██████████| 391/391 [04:46<00:00,  1.37it/s]
Val Epoch:0 | Loss 2.1793 | NatPrec1 78.940 | NatPrec5 99.230 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:01<00:00, 48.58it/s]
Val Epoch:0 | Loss 2.2114 | AdvPrec1 52.820 | AdvPrec5 92.430 | Reg term: 0.0 ||: 100%|██████████| 79/79 [00:54<00:00,  1.45it/s]
Train Epoch:0 | Loss 2.1649 | NatPrec1 85.046 | NatPrec5 99.668 | Reg term: 0.0 ||: 100%|██████████| 391/391 [00:21<00:00, 17.95it/s]
Val Epoch:0 | Loss 2.1570 | NatPrec1 81.080 | NatPrec5 99.270 | Reg term: 0.0 

In [14]:
import torch.nn as nn
class Wrapper(nn.Module):
    def __init__(self, base_model):
        super().__init__()
        self.base_model = base_model
    def forward(self, x):
        output = self.base_model(x)
        #print(type(output[0]))
        return output[0]
net = Wrapper(m)

# We use cox (http://github.com/MadryLab/cox) to log, store and analyze
# results. Read more at https//cox.readthedocs.io.
from cox.utils import Parameters
import cox.store

# Hard-coded dataset, architecture, batch size, workers
ds = CIFAR('/path/to/cifar')
#m, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/train_out/checkpoint.pt.latest")
#m.model.linear = CosineLayer(2048, 10)
#m.attacker.model.linear = CosineLayer(2048, 10)
m2, _ = model_utils.make_and_restore_model(arch='resnet50', dataset=ds, resume_path="/workspace/cifar_linf_8.pt")
net = Wrapper(m)
net2 = Wrapper(m2)

=> loading checkpoint '/workspace/cifar_linf_8.pt'
=> loaded checkpoint '/workspace/cifar_linf_8.pt' (epoch 153)


In [15]:
def evaluate_attacked_model(loader, phase, atk, limit=5000, model=net):
    correct = 0
    total = 0
    counter = 0
    total_loss = 0.0
    for data in loader:
        images, labels = data[0].to(device), data[1].to(device)
        # calculate outputs by running images through the network
        #print(labels)
        net.train()
        images = atk(images, labels)
        net.eval()
        outputs = model(images)
        #print(outputs.shape)
        # the class with the highest energy is what we choose as prediction
        #print(outputs.data)
        _, predicted = torch.max(outputs.data, 1)
        loss = criterion(outputs, labels)
        total_loss += loss.item()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        if total >= limit:
            break
    print(f'{phase} accuracy: {100 * correct / total} ({correct} / {total}) %')
    return correct, total_loss / math.ceil(limit / 16)

In [18]:
import torchattacks
atk = torchattacks.PGD(net)
#atk = torchattacks.AutoAttack(net)
#atk.set_normalization_used((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
atk2 = torchattacks.PGD(net2)
#atk2 = torchattacks.AutoAttack(net2)
import torch
import math
criterion = nn.CrossEntropyLoss()
device = torch.device('cuda')
evaluate_attacked_model(val_loader, "test_atk", atk, limit=10000, model=net)
evaluate_attacked_model(val_loader, "test_atk", atk2, limit=10000, model=net2)

test_atk accuracy: 54.53 (5453 / 10000) %
test_atk accuracy: 49.88 (4988 / 10000) %


(4988, 0.17665645103454589)

In [23]:
from typing import Any, Callable, Dict, Mapping, Optional, Sequence, Tuple
from typing_extensions import OrderedDict

In [24]:
import tables

ImportError: cannot import name 'Callable' from 'typing_extensions' (/usr/local/lib/python3.10/dist-packages/typing_extensions.py)